In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import numpy as np
import requests, zipfile, io #zum abspeichern von PDFs
from tqdm import tqdm
import PyPDF2
import re
from os import listdir
from os.path import isfile, join

## URL von allen Hauptseiten holen

In [ ]:
#alle urls und infos der Hauptseiten holen.
url = "https://kr-geschaefte.zug.ch/gast/geschaefte?commit=Filtern&geschaeft_filter%5Babgeschlossen_bis%5D=&geschaeft_filter%5Babgeschlossen_von%5D=&geschaeft_filter%5Barten_refs%5D%5B%5D=&geschaeft_filter%5Beingereicht_bis%5D=&geschaeft_filter%5Beingereicht_von%5D=&geschaeft_filter%5Bfrist_bis%5D=&geschaeft_filter%5Bhistorische_staende_refs%5D%5B%5D=&geschaeft_filter%5Bkommissionen_refs%5D%5B%5D=&geschaeft_filter%5Bstaende_refs%5D%5B%5D=&geschaeft_filter%5Bstatus_ids%5D%5B%5D=haengig&geschaeft_filter%5Bstatus_ids%5D%5B%5D=abgeschlossen&geschaeft_filter%5Bstatus_ids%5D%5B%5D=&geschaeft_filter%5Btitel%5D=&geschaeft_filter%5Bzustaendig_refs%5D%5B%5D=&page="

alle_seiten = []
for seite in range(1,41):
    r=requests.get(url+str(seite))
    soup = BeautifulSoup(r.text,"html.parser")
    
    tr_list=soup.find_all("tr")[2:] #erst ab der Position 2 sind die Daten relevant.
    
    for element in tr_list:
        td_list = element.find_all("td")
        vorlag_nr=td_list[0].text
        href = td_list[0].find("a")["href"]
        text = td_list[1].text
        stand = td_list[2].text
        zustaendig = td_list[3].text
        geschaeft_art = td_list[4].text
        status = td_list[5].text
        
        mini_dict = {"VorlageNR":vorlag_nr,"url":href,"Geschäft":text, "Stand":stand, "Zuständigkeit":zustaendig, "Art des Geschäfts": geschaeft_art, "Status":status}

        alle_seiten.append(mini_dict)



In [ ]:
pd.DataFrame(alle_seiten)

In [2]:
df_vs=pd.read_csv("KRFrames/df_v_Hauptseiten")

In [3]:
df_vs

,Unnamed: 0,index,VorlageNR,url,Geschäft,Stand,Zuständigkeit,Art des Geschäfts,Status
0,0,0,3020,/gast/geschaefte/2008,Postulat von Luzian Franzini und Esther Haas b...,Überweisung an Regierungsrat,Sicherheitsdirektion,Postulat,hängig
1,1,1,3019,/gast/geschaefte/2007,"Motion von Luzian Franzini, Rita Hofer, Tabea ...",Überweisung an Regierungsrat,Direktion des Innern,Motion,hängig
2,2,2,3018,/gast/geschaefte/2006,"Interpellation von Fabio Iten, Laura Dittli un...",Überweisung an Regierungsrat,Volkswirtschaftsdirektion,Interpellation,hängig
3,3,3,3017,/gast/geschaefte/2005,"Postulat von Tabea Zimmermann Gibson, Stéphani...",Überweisung an Regierungsrat,Direktion des Innern,Postulat,hängig
4,4,6,3014,/gast/geschaefte/2002,Interpellation der SVP-Fraktion betreffend mob...,Kenntnisnahme,Sicherheitsdirektion,Interpellation,hängig
...,...,...,...,...,...,...,...,...,...
1215,1215,1946,762,/gast/geschaefte/1331,Motion der erweiterten Justizprüfungskommissio...,Datenmigration,NaN,Motion,abgeschlossen
1216,1216,1952,666,/gast/geschaefte/1325,Motion der Kommission Teilrevision Personalges...,Datenmigration,NaN,Motion,abgeschlossen
1217,1217,1968,304,/gast/geschaefte/1309,Motion von Manuela Weichelt betreffend HIV-Prä...,Datenmigration,NaN,Motion,abgeschlossen
1218,1218,1969,282,/gast/geschaefte/1712,Motion von Christoph Hohler betreffend Radstre...,Frist erstreckt,Baudirektion,Motion,hängig


# hier ein Test von Barnaby, wie man die Namen alleine mit "Split" separieren kann

- Vielleicht zuerst noch mit Pandas nur die Vorstösse raussuchen. Ist ja das, was mich interessiert.
- Rausfinden, wie ich die Namen nun rausziehe und in ein aufklappbares Dataframe stelle


In [4]:

def splitting(elem):
    elem = elem.split('betreffend')[0].strip()
    if 'von' in elem:
        elem = elem.split('von')[1].strip()
        elem = elem.replace(' und', ',')
    elif 'der' in elem:
        elem = elem.split('der')[1].strip()    
    return elem
df_vs["Geschäft"]=df_vs['Geschäft'].apply(splitting) #.apply ist um eine Funktion auf eine ganze Reihe Pandas einzusetze

# Nun rausfinden, wie ich ein DataFrame machen kann, das man aufklappt

In [5]:
#das funktionerit also nicht
def splitkomma(elem):
    elem=elem.split(", ")
    return elem


In [6]:
df_vs["Geschäft"]=df_vs["Geschäft"].apply(splitkomma) 

In [7]:

nameliste=df_vs["Geschäft"].tolist()

In [8]:
nameliste

[['Luzian Franzini', 'Esther Haas'],
 ['Luzian Franzini',
  'Rita Hofer',
  'Tabea Zimmermann Gibson',
  'Andreas Lustenberger',
  'Hanni Schriber-Neiger'],
 ['Fabio Iten', 'Laura Dittli', 'Anna Bieri'],
 ['Tabea Zimmermann Gibson', 'Stéphanie Vuichard', 'Esther Haas'],
 ['SVP-Fraktion'],
 ['SP-Fraktion'],
 ['Ivo Egger',
  'Stéphanie Vuichard',
  'Hanni Schriber-Neiger',
  'Andreas Lustenberger'],
 ['Stéphanie Vuichard', 'Mariann Hess', 'Rita Hofer'],
 ['Rainer Leemann', 'Michael Arnold'],
 ['SP-Fraktion'],
 ['Pirmin Andermatt', 'Heini Schmid'],
 ['CVP-Fraktion'],
 ['Manuela Leemann', 'Benny Elsener', 'Richard Rüegg'],
 ['Hans Baumgartner',
  'Manuela Käch',
  'Jean Luc Mösch',
  'Brigitte Wenzin Widmer',
  'Rainer Suter',
  'Thomas Gander',
  'Drin Alaj',
  'Petra Muheim Quick',
  'Claus Soltermann'],
 ['Pirmin Andermatt', 'Zari Dzaferi', 'Urs Andermatt'],
 ['FDP-Fraktion'],
 ['Andreas Lustenberger'],
 ['CVP-Fraktion'],
 ['FDP-Fraktion'],
 ['Anna Bieri', 'Barbara Häseli'],
 ['SP-Frakt

In [14]:
namenliste_n=[]
counter=0
for alleeintraege in nameliste:
    namens = 
   
    
        
          
    
    minidictnamen={"Name":namens}
    namenliste_n.append(minidictnamen)
  

NameError: name 'my_list' is not defined

In [ ]:
namenliste_n

In [ ]:
#Ich versuche es mal schriftlich bevor ich den Loop schreibe:
liste=[]
namenliste={"sophie", "anna","julia"}
minidicttest={"ART": "Motion", "NUMMER": 23455, "NAMEN":namenliste}
liste.append(minidicttest)

In [ ]:
df_test=pd.DataFrame(liste)

In [ ]:
df_test

In [ ]:
for key, value in minidict_g.items():
        if value == '':
            minidict_g[key] = 'NaN'  #hier schaue ich noch, dass ich die Leeren Zeilen mit NAN ersetzten kann.
        else:
            minidict_g[key] = value